In [2]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../')

# Imports 

In [60]:
### Classes
from src.DataLoader import DataLoader
from src.Tensor import *
from interface.dataLoader_UI import Load_datas_pred
from interface.dataLoader_UI import Datas_truncature

import numpy as np
import os
import pandas as pd
import sys


In [ ]:
file_path = "\\wsl.localhost\Ubuntu\home\marie\code\marieyalap\Project\Traian_Project\Datas\Datas_for_pred\test\test\Traian_walk_fresh"

In [1]:
pwd

'/home/marie/code/CapnJackSparrow/09-IMU-based-fatigue-detection/notebooks'

In [11]:
csv_path = "../../../marieyalap/Project/Traian_Project/Datas/Datas_for_pred/test/test/Traian_walk_fresh"

In [12]:
data_pred = Load_datas_pred(csv_path)

/home/marie/code/CapnJackSparrow/09-IMU-based-fatigue-detection/notebooks/../interface/dataLoader_UI.py:5: DtypeWarning: Columns (0,1,2,3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f"{csv_file_path}/Accelerometer.csv", header=None)
/home/marie/code/CapnJackSparrow/09-IMU-based-fatigue-detection/notebooks/../interface/dataLoader_UI.py:24: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df['UNIX_time'] = pd.to_datetime(df['UNIX_time'], unit='s')


In [28]:
data_pred

,Seconds_elapsed,Z_acceleration,Y_acceleration,X_acceleration
UNIX_time,,,,
2024-05-26 01:32:35.625000,-3.848350,-0.051010,0.000545,0.032262
2024-05-26 01:32:35.656250,-3.818350,-0.032744,-0.040288,-0.002768
2024-05-26 01:32:35.687500,-3.788350,0.080884,-0.128668,-0.154711
2024-05-26 01:32:35.718750,-3.753350,-0.272538,0.050467,0.013127
2024-05-26 01:32:35.750000,-3.718350,-0.288374,0.045536,-0.088909
...,...,...,...,...
2024-05-26 02:12:19.812500,2380.336676,-0.579128,-0.243219,-1.272636
2024-05-26 02:12:19.843750,2380.371589,0.125345,-0.725636,-0.573263
2024-05-26 02:12:19.875000,2380.406504,-0.398860,-1.167349,0.024975


In [ ]:
### 1) changing the order of columns => X, Y,Z => same names as the one from Data Loader
### 2) Find the middle and the number of lines in the datasets
### 3) truncatenate the datas in order to have 90 sec

In [20]:
#  df = df[cols]

#  cols = cols[-1:] + cols[:-1]

cols = data_pred.columns.tolist()
cols

['Seconds_elapsed', 'Z_acceleration', 'Y_acceleration', 'X_acceleration']

In [35]:
### 1) changing the order of columns => X, Y,Z => same names as the one from Data Loader

datas_pred_ordered = data_pred[["X_acceleration","Y_acceleration","Z_acceleration"]]
datas_pred_ordered

,X_acceleration,Y_acceleration,Z_acceleration
UNIX_time,,,
2024-05-26 01:32:35.625000,0.032262,0.000545,-0.051010
2024-05-26 01:32:35.656250,-0.002768,-0.040288,-0.032744
2024-05-26 01:32:35.687500,-0.154711,-0.128668,0.080884
2024-05-26 01:32:35.718750,0.013127,0.050467,-0.272538
2024-05-26 01:32:35.750000,-0.088909,0.045536,-0.288374
...,...,...,...
2024-05-26 02:12:19.812500,-1.272636,-0.243219,-0.579128
2024-05-26 02:12:19.843750,-0.573263,-0.725636,0.125345
2024-05-26 02:12:19.875000,0.024975,-1.167349,-0.398860


In [39]:
### 2) Find the middle and the number of lines in the datasets
len_datas = datas_pred_ordered.shape[0]
len_datas

76299

In [48]:
time_laps = 90
nb_event_time_laps = 32*time_laps
nb_event_time_laps
middle_target_time = np.round(len_datas/2)
middle_target_time

38150.0

In [56]:
begin_index_truncated = int(middle_target_time - (nb_event_time_laps/2))
begin_index_truncated

36710

In [55]:
end_index_truncated = int(middle_target_time + (nb_event_time_laps/2))
end_index_truncated

39590

In [53]:
## verification :
end_index_truncated - begin_index_truncated == nb_event_time_laps

True

In [47]:
nb_event_time_laps

2880

In [ ]:
### 3) truncatenate the datas in order to have 90 sec

In [59]:
df_pred_truncated = datas_pred_ordered.iloc[begin_index_truncated:end_index_truncated,:]
df_pred_truncated

,X_acceleration,Y_acceleration,Z_acceleration
UNIX_time,,,
2024-05-26 01:51:42.812500,0.180846,-1.581176,-0.194265
2024-05-26 01:51:42.843750,0.352007,-2.538964,-0.464634
2024-05-26 01:51:42.875000,0.453938,-3.033417,-0.205182
2024-05-26 01:51:42.906250,0.449854,-3.368692,0.028456
2024-05-26 01:51:42.937500,0.352493,-3.132666,0.295715
...,...,...,...
2024-05-26 01:53:12.656250,0.105730,-0.115864,1.141715
2024-05-26 01:53:12.687500,0.181555,3.696345,1.003456
2024-05-26 01:53:12.718750,-0.664856,6.125153,1.758174


In [63]:
## Test the function :
data_pred = Datas_truncature(Load_datas_pred(csv_path),90)
data_pred

/home/marie/code/CapnJackSparrow/09-IMU-based-fatigue-detection/notebooks/../interface/dataLoader_UI.py:6: DtypeWarning: Columns (0,1,2,3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  # Remove the first row (index 0)
/home/marie/code/CapnJackSparrow/09-IMU-based-fatigue-detection/notebooks/../interface/dataLoader_UI.py:25: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  # Set the unix time as dataframe index


,X_acceleration,Y_acceleration,Z_acceleration
UNIX_time,,,
2024-05-26 01:51:42.812500,0.180846,-1.581176,-0.194265
2024-05-26 01:51:42.843750,0.352007,-2.538964,-0.464634
2024-05-26 01:51:42.875000,0.453938,-3.033417,-0.205182
2024-05-26 01:51:42.906250,0.449854,-3.368692,0.028456
2024-05-26 01:51:42.937500,0.352493,-3.132666,0.295715
...,...,...,...
2024-05-26 01:53:12.656250,0.105730,-0.115864,1.141715
2024-05-26 01:53:12.687500,0.181555,3.696345,1.003456
2024-05-26 01:53:12.718750,-0.664856,6.125153,1.758174
